<a href="https://colab.research.google.com/github/aytha-rajesh/lakehead/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Libraries


In [3]:
# Import the pandas library to read our dataset
import pandas as pd

# Get the train/test split package from sklearn for preparing our dataset to
# train and test the model with
from sklearn.model_selection import train_test_split

# Import the numpy library to work with and manipulate the data
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from keras.utils import to_categorical

import nltk
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Using TensorFlow backend.


Loading the Data


In [0]:
# import data from google drive
#data = pd.read_csv('train.tsv', sep='\t')
data = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')

Removing Stopwords

In [5]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 

pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


Text Vectorization

In [0]:
X = data.cleaned_reviews.values
Y = to_categorical(data.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer(max_features=4000,stop_words = None, ngram_range=(1,2))
X = vectorizer.fit_transform(X)



Splitting Data in 70:30 ratio


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2003)

Converting dataset to numpy array


In [0]:
import numpy as np
#convert the train data
x_train_np = x_train.toarray()
y_train_np = np.array(y_train)
#convert the test data
x_test_np = x_test.toarray()
y_test_np = np.array(y_test)

In [0]:
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

Importing Libraries for Creating the Network


In [10]:
#Import the pytorch library
import torch
print(torch.__version__)
#Import the 1Dconvolution layer
#Since we’re inputting a 1-dimensional row of data, we can't use 2D
from torch.nn import Conv1d
#Import the maxpooling layer
from torch.nn import MaxPool1d
#import flatten layer
from torch.nn import Flatten
#import linear layer
from torch.nn import Linear
#importing RELU activation function
from torch.nn.functional import relu
#importing sigmoid function
from torch import sigmoid
#Import the DataLoader and TensorDataset libraries from PyTorch
#to work with our datasets
from torch.utils.data import DataLoader, TensorDataset 

1.4.0


Network Creation

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()

def cnn(data_x):
  #Convolution layer 1
  model.add(Conv1D(filters = 64, kernel_size=2, activation='relu', input_shape=(data_x.shape[1],1)))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  #Convolution layer 2
  model.add(Conv1D(filters = 128, kernel_size=2, activation='relu'))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  model.add(Flatten())
  #Fully connected layer
  model.add(Dense(100, activation='relu'))
  #Dropout layer
  model.add(Dropout(0.2))
  #Output Layer
  model.add(Dense(5, activation='softmax')) 






Defining Performance Measures

In [0]:
from keras import metrics
from sklearn.metrics import f1_score
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:

cnn(x_train_np)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_m,precision_m, recall_m])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [15]:
history = model.fit(x_train, y_train_np, epochs=5, batch_size=64)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





109242/109242 [==============================] - 121s 1ms/step - loss: 1.0683 - categorical_accuracy: 0.5728 - f1_m: 0.4890 - precision_m: 0.6710 - recall_m: 0.3908
Epoch 2/10
109242/109242 [==============================] - 100s 919us/step - loss: 0.9323 - categorical_accuracy: 0.6291 - f1_m: 0.5823 - precision_m: 0.6995 - recall_m: 0.5002
Epoch 3/10
109242/109242 [==============================] - 100s 916us/step - loss: 0.8689 - categorical_accuracy: 0.6556 - f1_m: 0.6226 - precision_m: 0.7124 - recall_m: 0.5542
Epoch 4/10
109242/109242 [==============================] - 100s 918us/step - loss: 0.8194 - categorical_accuracy: 0.6758 - f1_m: 0.6510 - precision_m: 0.7236 - recall_m: 0.5927
Epoch 5/10
109242/109242 [==============================] - 100s 919us/step - loss: 0.7805 - categorical_accuracy: 0.6892 - f1_m: 0.6695 - precision_m: 0.7317 - recall_m: 0.6179
Epoch 6/10

In [16]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.6381733521295229:
f1_score = 0.6268411223293293
Precision = 0.6588675093277032
Recall = 0.5986586355675168


Saving the Model

In [17]:
#saving model to disk
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


Loading model and displaying summary

In [20]:

# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
 
# load model
#model = load_model('model.h5')
load_model('model.h5', custom_objects={'f1_m': f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
# summarize model.
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3999, 64)          192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1999, 64)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1998, 128)         16512     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 999, 128)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 127872)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12787300  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)              